test model from clean samples from various speakers

https://huggingface.co/models?other=voice-activity-detection

# Hugging Face Model Set up

In [3]:
pip install -U "huggingface_hub[cli]" datasets

Note: you may need to restart the kernel to use updated packages.


In [ ]:
! huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 

In [ ]:
from google.colab import userdata
HF_TOKEN = userdata.get('HF-Voizk')

# Model 1 : PyAnnote Voice activity detection


In [4]:
pip install pyannote.audio


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.0/811.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 824.8/824.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 17.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 38.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━

In [ ]:
from pyannote.audio import Pipeline
pipeline = Pipeline.from_pretrained("pyannote/voice-activity-detection",
                                    use_auth_token=HF_TOKEN)
output = pipeline("test.wav")


In [ ]:
print(output)

# Model 2 : Silero VAD

In [ ]:
import torch
torch.set_num_threads(1)

model, utils = torch.hub.load(repo_or_dir='snakers4/silero-vad', model='silero_vad')
(get_speech_timestamps, _, read_audio, _, _) = utils

wav = read_audio('test.wav')
speech_timestamps = get_speech_timestamps(
  wav,
  model,
  return_seconds=True,  # Return speech timestamps in seconds (default is samples)
)

In [ ]:
print(speech_timestamps)

# Model 3 : Wave2Vec2

In [9]:
pip install librosa soundfile

Note: you may need to restart the kernel to use updated packages.


In [5]:
# Let's see how to retrieve time steps for a model
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC
from datasets import load_dataset
import datasets
import torch

# import model, feature extractor, tokenizer
model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-base-960h")
tokenizer = AutoTokenizer.from_pretrained("facebook/wav2vec2-base-960h")
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

# load first sample of English common_voice
dataset = load_dataset("mozilla-foundation/common_voice_11_0", "en", split="train", streaming=True, trust_remote_code=True)
dataset = dataset.cast_column("audio", datasets.Audio(sampling_rate=16_000))
dataset_iter = iter(dataset)
sample = next(dataset_iter)

# forward sample through model to get greedily predicted transcription ids
input_values = feature_extractor(sample["audio"]["array"], return_tensors="pt").input_values
logits = model(input_values).logits[0]
pred_ids = torch.argmax(logits, axis=-1)

# retrieve word stamps (analogous commands for `output_char_offsets`)
outputs = tokenizer.decode(pred_ids, output_word_offsets=True)
# compute `time_offset` in seconds as product of downsampling ratio and sampling_rate
time_offset = model.config.inputs_to_logits_ratio / feature_extractor.sampling_rate

word_offsets = [
    {
        "word": d["word"],
        "start_time": round(d["start_offset"] * time_offset, 2),
        "end_time": round(d["end_offset"] * time_offset, 2),
    }
    for d in outputs.word_offsets
]
# compare word offsets with audio `en_train_0/common_voice_en_19121553.mp3` online on the dataset viewer:
# https://huggingface.co/datasets/mozilla-foundation/common_voice_11_0/viewer/en
word_offsets[:3]

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Reading metadata...: 948736it [00:19, 49804.39it/s]
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


[{'word': 'THE', 'start_time': np.float64(0.7), 'end_time': np.float64(0.78)},
 {'word': 'TRICK',
  'start_time': np.float64(0.88),
  'end_time': np.float64(1.08)},
 {'word': 'APPEARS',
  'start_time': np.float64(1.2),
  'end_time': np.float64(1.64)}]

#  Model 4 : WebRTC

The WebRTC VAD only accepts 16-bit mono PCM audio, sampled at 8000, 16000, 32000 or 48000 Hz. A frame must be either 10, 20, or 30 ms in duration:

In [ ]:
pip install webrtcvad


In [ ]:
import collections
import contextlib
import sys
import wave

import webrtcvad


def read_wave(path):
    """Reads a .wav file.

    Takes the path, and returns (PCM audio data, sample rate).
    """
    with contextlib.closing(wave.open(path, 'rb')) as wf:
        num_channels = wf.getnchannels()
        assert num_channels == 1
        sample_width = wf.getsampwidth()
        assert sample_width == 2
        sample_rate = wf.getframerate()
        assert sample_rate in (8000, 16000, 32000, 48000)
        pcm_data = wf.readframes(wf.getnframes())
        return pcm_data, sample_rate


def write_wave(path, audio, sample_rate):
    """Writes a .wav file.

    Takes path, PCM audio data, and sample rate.
    """
    with contextlib.closing(wave.open(path, 'wb')) as wf:
        wf.setnchannels(1)
        wf.setsampwidth(2)
        wf.setframerate(sample_rate)
        wf.writeframes(audio)


class Frame(object):
    """Represents a "frame" of audio data."""
    def __init__(self, bytes, timestamp, duration):
        self.bytes = bytes
        self.timestamp = timestamp
        self.duration = duration


def frame_generator(frame_duration_ms, audio, sample_rate):
    """Generates audio frames from PCM audio data.

    Takes the desired frame duration in milliseconds, the PCM data, and
    the sample rate.

    Yields Frames of the requested duration.
    """
    n = int(sample_rate * (frame_duration_ms / 1000.0) * 2)
    offset = 0
    timestamp = 0.0
    duration = (float(n) / sample_rate) / 2.0
    while offset + n < len(audio):
        yield Frame(audio[offset:offset + n], timestamp, duration)
        timestamp += duration
        offset += n


def vad_collector(sample_rate, frame_duration_ms,
                  padding_duration_ms, vad, frames):
    """Filters out non-voiced audio frames.

    Given a webrtcvad.Vad and a source of audio frames, yields only
    the voiced audio.

    Uses a padded, sliding window algorithm over the audio frames.
    When more than 90% of the frames in the window are voiced (as
    reported by the VAD), the collector triggers and begins yielding
    audio frames. Then the collector waits until 90% of the frames in
    the window are unvoiced to detrigger.

    The window is padded at the front and back to provide a small
    amount of silence or the beginnings/endings of speech around the
    voiced frames.

    Arguments:

    sample_rate - The audio sample rate, in Hz.
    frame_duration_ms - The frame duration in milliseconds.
    padding_duration_ms - The amount to pad the window, in milliseconds.
    vad - An instance of webrtcvad.Vad.
    frames - a source of audio frames (sequence or generator).

    Returns: A generator that yields PCM audio data.
    """
    num_padding_frames = int(padding_duration_ms / frame_duration_ms)
    # We use a deque for our sliding window/ring buffer.
    ring_buffer = collections.deque(maxlen=num_padding_frames)
    # We have two states: TRIGGERED and NOTTRIGGERED. We start in the
    # NOTTRIGGERED state.
    triggered = False

    voiced_frames = []
    for frame in frames:
        is_speech = vad.is_speech(frame.bytes, sample_rate)

        sys.stdout.write('1' if is_speech else '0')
        if not triggered:
            ring_buffer.append((frame, is_speech))
            num_voiced = len([f for f, speech in ring_buffer if speech])
            # If we're NOTTRIGGERED and more than 90% of the frames in
            # the ring buffer are voiced frames, then enter the
            # TRIGGERED state.
            if num_voiced > 0.9 * ring_buffer.maxlen:
                triggered = True
                sys.stdout.write('+(%s)' % (ring_buffer[0][0].timestamp,))
                # We want to yield all the audio we see from now until
                # we are NOTTRIGGERED, but we have to start with the
                # audio that's already in the ring buffer.
                for f, s in ring_buffer:
                    voiced_frames.append(f)
                ring_buffer.clear()
        else:
            # We're in the TRIGGERED state, so collect the audio data
            # and add it to the ring buffer.
            voiced_frames.append(frame)
            ring_buffer.append((frame, is_speech))
            num_unvoiced = len([f for f, speech in ring_buffer if not speech])
            # If more than 90% of the frames in the ring buffer are
            # unvoiced, then enter NOTTRIGGERED and yield whatever
            # audio we've collected.
            if num_unvoiced > 0.9 * ring_buffer.maxlen:
                sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
                triggered = False
                yield b''.join([f.bytes for f in voiced_frames])
                ring_buffer.clear()
                voiced_frames = []
    if triggered:
        sys.stdout.write('-(%s)' % (frame.timestamp + frame.duration))
    sys.stdout.write('\n')
    # If we have any leftover voiced audio when we run out of input,
    # yield it.
    if voiced_frames:
        yield b''.join([f.bytes for f in voiced_frames])

audio, sample_rate = read_wave("test.wav")
vad = webrtcvad.Vad()
frames = frame_generator(30, audio, sample_rate)
frames = list(frames)
segments = vad_collector(sample_rate, 30, 300, vad, frames)
for i, segment in enumerate(segments):
    path = 'chunk-%002d.wav' % (i,)
    print(' Writing %s' % (path,))
    write_wave(path, segment, sample_rate)

# Wav2Vec result from VoxCeleb Sample

In [ ]:
# Let's see how to retrieve time steps for a model
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC
from datasets import load_dataset
import datasets
import torch
import librosa

# import model, feature extractor, tokenizer
model = AutoModelForCTC.from_pretrained("facebook/wav2vec2-base-960h")
tokenizer = AutoTokenizer.from_pretrained("facebook/wav2vec2-base-960h")
feature_extractor = AutoFeatureExtractor.from_pretrained("facebook/wav2vec2-base-960h")

def wav2vec2_processor(file_path):
    audio_input, _ = librosa.load(file_path, sr=16_000)
    # forward sample through model to get greedily predicted transcription ids    
    input_values = feature_extractor(audio_input, return_tensors="pt").input_values
    # retrieve word stamps (analogous commands for `output_char_offsets`)
    logits = model(input_values).logits[0]
    pred_ids = torch.argmax(logits, axis=-1)
    outputs = tokenizer.decode(pred_ids, output_word_offsets=True)
    # compute `time_offset` in seconds as product of downsampling ratio and sampling_rate
    time_offset = model.config.inputs_to_logits_ratio / feature_extractor.sampling_rate

    word_offsets = [
        {
            "word": d["word"],
            "start_time": round(d["start_offset"] * time_offset, 2),
            "end_time": round(d["end_offset"] * time_offset, 2),
        }
        for d in outputs.word_offsets
    ]
    return word_offsets

# test
output_1 = wav2vec2_processor("voxceleb/TomCruiseSpeaks_sample1.wav")
output_2 = wav2vec2_processor("voxceleb/TomCruiseSpeaks_sample2.wav")
output_3 = wav2vec2_processor("voxceleb/TomCruiseSpeaks_sampleshort.wav")

print("Output Sample 1: ", output_1)
print("Output Sample 2: ", output_2)
print("Output Sample Short: ", output_3)


Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.
It is strongly recommended to pass the ``sampling_rate`` argument to this function. Failing to do so can result in silent errors that might be hard to debug.


Output Sample 1:  [{'word': 'AT', 'start_time': 0.34, 'end_time': 0.38}, {'word': 'THE', 'start_time': 0.44, 'end_time': 0.54}, {'word': 'END', 'start_time': 0.62, 'end_time': 0.7}, {'word': 'OF', 'start_time': 0.78, 'end_time': 0.82}, {'word': 'IT', 'start_time': 0.92, 'end_time': 0.98}, {'word': 'YOU', 'start_time': 1.08, 'end_time': 1.2}, {'word': 'YOU', 'start_time': 2.08, 'end_time': 2.16}, {'word': 'WANT', 'start_time': 2.24, 'end_time': 2.38}, {'word': 'TO', 'start_time': 2.4, 'end_time': 2.46}, {'word': 'GO', 'start_time': 2.5, 'end_time': 2.6}, {'word': 'OUT', 'start_time': 2.74, 'end_time': 2.82}, {'word': 'AND', 'start_time': 2.88, 'end_time': 2.94}, {'word': 'YOU', 'start_time': 2.96, 'end_time': 3.04}, {'word': 'WANT', 'start_time': 3.1, 'end_time': 3.2}]
Output Sample 2:  [{'word': 'YE', 'start_time': 0.2, 'end_time': 0.3}, {'word': 'I', 'start_time': 0.42, 'end_time': 0.44}, {'word': 'LOVE', 'start_time': 0.54, 'end_time': 0.7}, {'word': 'THESE', 'start_time': 0.74, 'end

### Output Sample 1: VoxCeleb/TomCruiseSpeaks_sample1.wav

[{'word': 'AT', 'start_time': 0.34, 'end_time': 0.38}, {'word': 'THE', 'start_time': 0.44, 'end_time': 0.54}, {'word': 'END', 'start_time': 0.62, 'end_time': 0.7}, {'word': 'OF', 'start_time': 0.78, 'end_time': 0.82}, {'word': 'IT', 'start_time': 0.92, 'end_time': 0.98}, {'word': 'YOU', 'start_time': 1.08, 'end_time': 1.2}, {'word': 'YOU', 'start_time': 2.08, 'end_time': 2.16}, {'word': 'WANT', 'start_time': 2.24, 'end_time': 2.38}, {'word': 'TO', 'start_time': 2.4, 'end_time': 2.46}, {'word': 'GO', 'start_time': 2.5, 'end_time': 2.6}, {'word': 'OUT', 'start_time': 2.74, 'end_time': 2.82}, {'word': 'AND', 'start_time': 2.88, 'end_time': 2.94}, {'word': 'YOU', 'start_time': 2.96, 'end_time': 3.04}, {'word': 'WANT', 'start_time': 3.1, 'end_time': 3.2}]


### Output Sample 2:  VoxCeleb/TomCruiseSpeaks_sample2.wav

[{'word': 'YE', 'start_time': 0.2, 'end_time': 0.3}, {'word': 'I', 'start_time': 0.42, 'end_time': 0.44}, {'word': 'LOVE', 'start_time': 0.54, 'end_time': 0.7}, {'word': 'THESE', 'start_time': 0.74, 'end_time': 0.9}, {'word': 'CEREMONIES', 'start_time': 1.36, 'end_time': 1.84}, {'word': 'IRE', 'start_time': 1.88, 'end_time': 1.96}, {'word': 'MEMBERS', 'start_time': 1.98, 'end_time': 2.24}, {'word': 'A', 'start_time': 2.28, 'end_time': 2.32}, {'word': 'LITTLE', 'start_time': 2.34, 'end_time': 2.52}, {'word': 'KID', 'start_time': 2.56, 'end_time': 2.8}, {'word': 'I', 'start_time': 2.94, 'end_time': 2.96}, {'word': 'USED', 'start_time': 3.06, 'end_time': 3.18}, {'word': 'TO', 'start_time': 3.24, 'end_time': 3.32}]


### Output Sample Short:  VoxCeleb/TomCruiseSpeaks_sampleshort.wav

[{'word': 'ONE', 'start_time': 0.06, 'end_time': 0.14}, {'word': 'TO', 'start_time': 0.16, 'end_time': 0.22}, {'word': 'EMBRACE', 'start_time': 0.3, 'end_time': 0.62}, {'word': 'LIFE', 'start_time': 0.74, 'end_time': 0.98}]